# Society of Mind using Spiking Agents

In [1]:
import torch
import numpy as np
import numpy.random as rd
import time
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import importlib


import matplotlib.pyplot as plt


import copy
from torch.optim.lr_scheduler import StepLR

from tqdm import tqdm
#from tqdm.notebook import tqdm, trange

from torchviz import make_dot

In [2]:
from models import *
from decision import *
from training import *
from utils_diverse import *
from spiking_models import *
from datasets_and_tasks import*

In [3]:
sys.path.append("/home/gb21/Code/SNNs/randman/")
from randman_snn import *
sys.path.append('/home/gb21/Code/SNNs/Recurrent SNNs/')
import rec_Layer
from rec_Layer import *

In [4]:
DCLL_path = '/home/gb21/Code/SNNs/DCLL/decolle-public/'
sys.path.append(DCLL_path)
sys.path.append(DCLL_path + 'decolle')
import snn_utils

In [5]:
DeepR_path = '/home/gb21/Code/SNNs/DEEP-R/'
import sys, os
sys.path.append(DeepR_path)
import deepR_torch as deepR

In [6]:
import warnings
#warnings.filterwarnings('ignore')

In [7]:
%load_ext autoreload
#%autoreload 2

In [8]:
loss_SL1 = torch.nn.SmoothL1Loss()   
cross_entropy = torch.nn.CrossEntropyLoss()

def reg_loss(s, alpha=5e-7) : 
    reg_loss = 0
    for i in range(len(s)) : 
        reg_loss += alpha*torch.sum(s[i])
        reg_loss += alpha*torch.mean(torch.sum(s[i],dim=0)**2)
        
    return reg_loss

In [9]:
def check_grad(model) : 
    for n, p in model.named_parameters() : 
        if p.grad is not None : 
            if (p.grad == 0).all() : 
                print(f'{n}, Zero Grad')
        elif p.requires_grad : print(f'{n}, None Grad')
            
def decision_making(rt, decison) : 
    try : 
        deciding_ag = int(decision)
        rt = rt[:, deciding_ag, ...]
    except ValueError : 
        if decision == 'max' : 
            rt = torch.stack([max_decision(r)[0] for r in rt])
        elif decision == 'sum' : 
            rt = rt[:, 0, ...] + rt[:, 1, ...]
    return rt

## MNIST

### Loading MNIST data as spiketrains

The following function will load the MNIST dataset using torchvision modules. It will download and pre-pre-process the data for faster usage.

In [14]:
gen_train = snn_utils.get_mnist_loader(256, Nparts=50, train=True, data_path='mnist/')
gen_test = snn_utils.get_mnist_loader(256, Nparts=50, train=False, data_path='mnist/')

Because MNIST is an image, we need to transform it into a spiketrain. The function __image2spiketrain__ in snn_utils takes case of this.

In [11]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 256
T = 50
burnin = 10
train_kwargs = {'batch_size': batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 0,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    
def iter_mnist(train = True, batchsize=batch_size, Nparts=10, T=T, max_rate=20, pool=False, double=False):
    if double : 
        double_dataset = DoubleMNIST(asym=True, train=train)
        subset = torch.utils.data.Subset(double_dataset, np.arange(len(double_dataset)//Nparts))
        gen = torch.utils.data.DataLoader(subset, **train_kwargs)
        n_classes = 100
    else : 
        gen = snn_utils.get_mnist_loader(batchsize, Nparts=Nparts, train=train, data_path='mnist/')
        n_classes = 10
        
    datait = iter(gen)
    for raw_input, raw_labels in datait:
        scale_factor = 2
        size = lambda input :(input.shape[2]//scale_factor, input.shape[3]//scale_factor)
        if pool : 
            raw_input = torch.nn.functional.interpolate(raw_input, size = size(raw_input), mode = 'bilinear')
            
        data, labels1h = snn_utils.image2spiketrain(raw_input, raw_labels, max_duration=T, gain=max_rate, n_classes=n_classes)
        data_t = torch.FloatTensor(data)
        labels_t = torch.Tensor(labels1h)
        #print(labels1h)
        
        yield data_t, labels_t 
            
def get_double_data(data, label, task='parity_digits') : 
    idxs = np.arange(data.shape[1])
    np.random.shuffle(idxs)
    double_data = torch.stack([data, data[:, idxs, ...]], axis=1)
    
    target = label.argmax(-1)
    double_target = 10*target + target[:, idxs, ...]
    
    double_target, n_classes = get_task_target(double_target, task)
    
    double_label = torch.nn.functional.one_hot(double_target, n_classes).float()
    
    return double_data, double_label


def get_task(label, task) : 
    double_target = label.argmax(-1)
    double_target, n_classes = get_task_target(double_target, task)
    double_label = torch.nn.functional.one_hot(double_target, n_classes).float()
    
    return double_label, n_classes

In [12]:
method = 'rtrl'
online_learning = (method == 'rtrl')

n_in = 784
n_hid = 100
n_out = 2
layer_type = LIFLayer
p_con = 0.25

agents = [SpikingAgent(n_in, [n_hid], n_out, str(n), method=method) for n in range(2)]
sparse_con = (torch.ones((2, 2)) - torch.eye(2))*p_con

community = SpikingCommunity(agents, sparse_con).to(device)

lr = 5e-7

optimizer_agents = torch.optim.Adamax(community.agents.parameters(), lr=lr, betas=[0., .95])

model = community

deepR_params = l1, gdnoise, lr, gamma, cooling = 1e-5, 0, lr, 0.99, 0.99
#gdnoise = lr*1e-6
deepR_params_dict = {'l1' : l1, 'gdnoise' : gdnoise, 'lr' : lr, 'gamma' : gamma, 'cooling' : cooling}
optimizer_connections = torch.optim.Adam(community.connections.parameters(), lr=lr, betas=[0., .95])


In [12]:
n

NameError: name 'n' is not defined

In [ ]:
data, label = next(iter_mnist(double= True))
data_d, double_label_d = data.to(device), label.to(device)
label_d, _ = get_task(double_label_d, 'parity_digits')

In [ ]:
double_target = double_label_d.argmax(-1)
target = label_d.argmax(-1)
digits = get_digits(double_target)
(digits[0] == target).float().mean()

In [ ]:
make_dot(loss_tv, dict(community.named_parameters())).render('graphs/SNNCom')

In [ ]:
model = community.agents[0]
optimizer_agents = torch.optim.Adamax(model.parameters(), lr=1e-8, betas=[0., .95])


In [ ]:
community.agents[0].LIF_layers[0].sg_function

## Training

In [13]:
n_epochs = 50
task = 'parity'
decision = 'max'
save_graph = False
check_gradients = False

pbar = tqdm(range(n_epochs), desc='Training : ')
            
nb_reconnect = []
deciding_agents = []

train_accs, train_losses = [], []
test_accs, test_losses = [], []

descs = ['', '']
            
com_training = (type(model) is SpikingCommunity)

if com_training : 
    thetas_list, sparsity_list = [], []
    for c in model.connections.values() : 
        if type(c) is LIFLayerConnect : 
            c = c.base_layer
        if type(c) is deepR.Sparse_Connect : 
            con  = c
            thetas_list.append(con.thetas[0])
            sparsity_list.append(con.sparsity_list[0])
            
        else : 
            thetas_list, sparsity_list = [], []
        
for epoch in pbar : 
    
    #Training :
    for batch_idx, (data, label) in enumerate(iter_mnist(double=True)):
            
        model.train()
        data_d = data.to(device)
        label_d = label.to(device)
        
        if com_training : 
            #data_d, label_d = get_double_data(data_d, label_d, task=task)
            label_d, _ = get_task(label_d, task)
            ''
        
        model.init(data_d, burnin=burnin)
        
        loss_hist = 0
        readout = 0
        for n in range(burnin, T) : 
            rt, (st, ut) = model.forward(data_d[n])  
            
            if com_training : 
                rt = decision_making(rt, decision)                        
            
            loss_tv = torch.sum(torch.stack([loss_SL1(r, label_d[n]) for r in rt]))
            
            #loss_tv = cross_entropy(rt, label_d[n].argmax(-1))
            if online_learning : 
                if batch_idx == 0 and save_graph : make_dot(loss_tv, dict(community.named_parameters())).render('graphs/SNNCom_Online')
                loss_tv.backward()
                optimizer_agents.step()
                if n != burnin and check_gradients :
                    check_grad(model)
                optimizer_agents.zero_grad()
                
                if com_training : 
                    nb_new_con = step_connections_spiking(model, optimizer_connections, False, thetas_list,
                                                      sparsity_list, deepR_params_dict=deepR_params_dict)
                    nb_reconnect.append(nb_new_con.cpu().data.item())
                    optimizer_connections.zero_grad()
                    
            loss_hist += loss_tv
            readout += rt[-1]
        
        if not online_learning : 
            if batch_idx == 0 and save_graph : make_dot(loss_hist, dict(community.named_parameters())).render('graphs/SNNCom_Offline')
            loss_hist.backward()
            optimizer_agents.step()
            if check_gradients : check_grad(model)
            optimizer_agents.zero_grad()
            
            if com_training : 
                nb_new_con = step_connections_spiking(model, optimizer_connections, False, thetas_list,
                                                      sparsity_list, deepR_params_dict=deepR_params_dict)
                nb_reconnect.append(nb_new_con.cpu().data.item())
                optimizer_connections.zero_grad()
            
            
                    
        acc = (readout.argmax(axis=-1)==label_d[-1].argmax(axis=-1)).float()
        train_accs.append(acc.mean().cpu().item())
        train_losses.append(loss_hist.cpu().item())

        descs[0] = str('Train Epoch: {} , Loss: {:.3f}, Acc: {:.3f}'.format(
                            epoch, loss_hist.item(), acc.mean().item()))
        
        pbar.set_description(descs[0]+descs[1])
        
    #Testing : 
    with torch.no_grad() : 
        for batch_idx, (data, label) in enumerate(iter_mnist(train=False, double=True)):

            data_d = data.to(device)
            label_d = label.to(device)

            if com_training : 
                #data_d, label_d = get_double_data(data_d, label_d, task=task)
                label_d, _ = get_task(label_d, task)
                ''

            model.init(data_d, burnin=burnin)

            loss_hist = 0
            readout = 0
            for n in range(burnin, T) : 

                rt, (st, ut) = model.forward(data_d[n])  
                if com_training : 
                    rt = rt[:, 1, ...]
                    #rt = torch.stack([max_decision(r)[0] for r in rt])
                    #rt = rt[:, 0, ...] + rt[:, 1, ...]
                    #if len(deciding_ags) == batch_size : deciding_agents.append(deciding_ags.cpu().data)

                loss_tv = torch.sum(torch.stack([loss_SL1(r, label_d[n]) for r in rt]))
                loss_hist += loss_tv
                readout += rt[-1] 


            acc = (readout.argmax(axis=-1)==label_d[-1].argmax(axis=-1)).float()

            descs[1] = str('| Test : Loss: {:.3f}, Acc: {:.3f}'.format(
                                loss_hist.item(), acc.mean().item()))
            
            test_accs.append(acc.mean().cpu().item())
            test_losses.append(loss_hist.cpu().item())

            pbar.set_description(descs[0]+descs[1])
    


Train Epoch: 49 , Loss: 3.304, Acc: 0.786| Test : Loss: 4.554, Acc: 0.534: 100%|██████████| 50/50 [31:23<00:00, 37.67s/it]


In [14]:
community.connections

ModuleDict(
  (01): LIFLayerConnect(
    (base_layer): Sparse_Connect(
      (thetas): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
      (weight): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
      (signs): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
    )
  )
  (10): LIFLayerConnect(
    (base_layer): Sparse_Connect(
      (thetas): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
      (weight): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
      (signs): ParameterList(  (0): Parameter containing: [torch.cuda.FloatTensor of size 100x100 (GPU 0)])
    )
  )
)

In [ ]:
def check_0(state) : 
    checks = []
    if type(state) is list : 
        for s in state : 
            checks.append(check_0(s))
    else : 
        for s in state : 
            check = (s.data == 0).all()
            checks.append(check)
            
    return checks

check_0(community.agents_states)


In [ ]:
plt.plot(test_losses)


In [ ]:
plt.plot(test_accs)

In [ ]:
plt.plot(torch.mean(torch.stack(deciding_agents[299::300]).float(), axis=-1).cpu())
plt.title(f'Mean = {torch.mean(torch.stack(deciding_agents[299::300]).float())}')
plt.show()


## Randman

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 256

randman_params = nb_classes, nb_units, T, nb_samples, nb_spikes = 5, 50, 50, 288, 5
randman_data = make_spiking_dataset_torch(nb_classes=nb_classes, nb_units=nb_units, nb_spikes=nb_spikes, nb_steps=T, dim_manifold=1, nb_samples=nb_samples//nb_classes, classification=True, model_type='dcll')
nb_total = nb_samples*nb_classes
train_data, test_data = split_data(randman_data, p=0.89)
train_loader, test_loader = generator_samples_randman(train_data, batchsize=64), generator_samples_randman(test_data, batchsize=64)
data, target = next(iter(train_loader))